<a href="https://colab.research.google.com/github/MindEvo/Doggo-Moods/blob/main/ANN_DogEmotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####################
# IMPORT LIBRARIES #
####################
import os
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn import metrics
from sklearn.preprocessing import scale, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neural_network import MLPClassifier
%matplotlib inline

In [ ]:
#####################################
# Load label data from local drive  #
#####################################
local_path = ""
labels = pd.read_csv(local_path)

# Sort the labels
sorted_labels = labels.sort_values(by="filename")

############
# CHECK IT #
############
sorted_labels.head(20)

FileNotFoundError: [Errno 2] No such file or directory: 'F:/SchoolThings/CSULA/2023Spring/CS4662 - DeepLearning/CS4662_FinalProject/Dog_Emotion/labels.csv'

In [ ]:
###########################################################
# Determine average dimensions of the images for resizing #
###########################################################
# widths = []
# heights = []

# for file in os.listdir(path):
#     img_path = os.path.join(path, file)
#     img = cv2.imread(img_path)
#     height, width, _ = img.shape

#     widths.append(width)
#     heights.append(height)

# avg_width = sum(widths) / len(widths)
# avg_height = sum(heights) / len(heights)

# print(f"Average width: {avg_width}")
# print(f"Average height: {avg_height}")

In [ ]:
##############
# Load image #
##############
path = "YOUR FILEPATH HERE"
images = []

#Need to ensure the images are sorted
sorted_files = sorted(os.listdir(path))

#Load the images into an array
for fil in sorted_files:
    img = mpimg.imread(os.path.join(path, fil))
    images.append(img)

In [ ]:
############################
# Resize images to 384x384 #
############################
desired_shape = (192, 192)
resized_images = []
for img in images:
    resized_img = cv2.resize(img, desired_shape, interpolation=cv2.INTER_AREA)
    resized_images.append(resized_img)

In [ ]:
############
# Check it #
############
#print(sorted_files[3950:3999])
plt.imshow(resized_images[3999], cmap=plt.cm.gray)

In [ ]:
label_encoder = LabelEncoder()
#############################################
# Build the feature matrix and label vector #
#############################################
X = np.array([img.flatten() for img in resized_images])
y = label_encoder.fit_transform(labels.iloc[:, -1])

#####################
# Check data frames #
#####################
print(f"X Shape:{X.shape}\ny Shape:{y.shape}")

In [ ]:
##########################
# Normalize the features #
##########################
X_normal = scale(X)

In [ ]:
# Calculate the mean and standard deviation of the scaled features to verify normalization
mean = X_normal.mean(axis=0)
std_dev = X_normal.std(axis=0)

print("Mean of the scaled features:", mean)
print("Standard deviation of the scaled features:", std_dev)

In [ ]:
##################
# Create the ANN #
##################
my_ANN = MLPClassifier(hidden_layer_sizes=(16,16,16,8,8,8),
                       random_state=1,
                       learning_rate_init=0.02,
                       solver='sgd',
                       alpha=0.1,
                       verbose=True,
                       activation='relu',
                       tol=0.0002
                      )

####################
# Cross Validation #
####################
accuracy_list = cross_val_score(my_ANN, X_normal, y, cv=10, scoring='accuracy')

##########################
# Calculate the accuracy #
##########################
accuracy_cv = accuracy_list.mean()
print(f"\nAccuracy: {round(accuracy_cv, 4)}")

In [ ]:
##################################################################
# define a range for the "number of neurons" in the hidden layers #
# for a network with 4 hidden layers                              #
##################################################################
neuron_number = [(i, j, k) for i in range(4, 24, 4) for j in range(4, 24, 4) for k in range(4, 24, 4)]

##########################################
# create a dictionary for grid parameter #
##########################################
param_grid = dict(hidden_layer_sizes=neuron_number)
print(param_grid, '\n')

##########################
# instantiate the model: #
##########################
my_ANN = MLPClassifier(activation='relu', solver='sgd',
                       alpha=1, random_state=1,
                       learning_rate_init=0.02,
                       verbose=True, max_iter=200)

##################################
# Create the GridSearchCV object #
##################################
my_grid = GridSearchCV(my_ANN, param_grid, cv=10, scoring='accuracy')

###############
# Fit the grid #
###############
grid_result = my_grid.fit(X_normal, y)